### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script. We can use `KerasTunner` or `Hyperopt` packages.

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform


In [110]:
import keras
import numpy as np
import pandas as pd

from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature
import mlflow.tensorflow

In [111]:
import tensorflow as tf
print(f'Tensorflow version: {tf.__version__}\nKeras version: {keras.__version__}')

Tensorflow version: 2.18.0
Keras version: 3.8.0


In [112]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [113]:
data.quality.unique()

array([6, 5, 7, 8, 4, 3, 9])

In [114]:
## Split the data into training,validation and test sets
train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [115]:
train[['quality']].values.ravel()

# train[['quality']].values.squeeze()
# train[['quality']].values.flatten()

array([6, 6, 5, ..., 6, 6, 8])

- `ravel()` devuelve un array aplanado (1D) tratando de reutilizar la memoria original si es posible
- `flatten()` crea una copia explícita del array aplanado
- `squeeze()` elimina las dimensiones de tamaño 1 del array (ej: de (3,1,2) a (3,2)).

In [116]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)
signature=infer_signature(train_x,train_y)

In [117]:
train_x.shape[1], train_x.shape
# 2983 rows and 11 columns/features

(11, (2938, 11))

In [118]:
np.mean(train_x,axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [119]:
### ANN Model
from tensorflow.keras.layers import Dense, Input, Normalization
from tensorflow.keras import Sequential

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    # We use a normalization layer and not use the sklearn StandarScaler()
    # With this, we include the noramlization step in the tf pipeline.
    # With this we enable the GPU to make the transformation 
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)
    
    ## Define model architecture

    model=Sequential(
        [Input(shape = (train_x.shape[1],)),
         Normalization(mean=mean,variance=var), # We can do the same with the StandarScaler() of scikit-learn
         Dense(64,activation='relu'),
         Dense(1)
        ]
    )

    ## compile the model
    model.compile(loss=tf.keras.losses.mae,
                  optimizer=tf.keras.optimizers.SGD(learning_rate=params["lr"],
                                                     momentum=params["momentum"]),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])

    ## Train the ANN model with lr and momentum params with MLFLOW tracking
    signature=infer_signature(train_x,train_y)
    with mlflow.start_run(nested=True): 
        #! Nested= true permite organizar las ejecuciones (runs) en una jerarquía de padre-hijo
        #! Estructura en forma de árbol en la interfaz de usuario de MLflow, facilitando la agrupación de experimentos relacionados
        model.fit(train_x,
                  train_y,
                  validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=32)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,
                                   valid_y,
                                   batch_size=32)

        eval_rmse=eval_result[1]

        ## Log the parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model
        mlflow.tensorflow.log_model(model = model,
                                    artifact_path = "model",
                                    signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [120]:
# ### ANN Model

# def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

#     ## Define model architecture
#     mean=np.mean(train_x,axis=0)
#     var=np.var(train_x,axis=0)

#     model=keras.Sequential(
#         [
#             keras.Input([train_x.shape[1]]),
#             keras.layers.Normalization(mean=mean,variance=var),
#             keras.layers.Dense(64,activation='relu'),
#             keras.layers.Dense(1)

#         ]
#     )

#     ## compile the model
#     model.compile(optimizer=keras.optimizers.SGD(
#         learning_rate=params["lr"],momentum=params["momentum"]
#     ),
#     loss="mean_squared_error",
#     metrics=[keras.metrics.RootMeanSquaredError()]
#     )

#     ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
#     with mlflow.start_run(nested=True):
#         model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
#                   epochs=epochs,
#                   batch_size=64)
        
#         ## Evaluate the model
#         eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

#         eval_rmse=eval_result[1]

#         ## Log the parameters and results
#         mlflow.log_params(params)
#         mlflow.log_metric("eval_rmse",eval_rmse)

#         ## log the model

#         mlflow.tensorflow.log_model(model,"model",signature=signature)

#         return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [121]:
# We need to define this objective function to use it with hyperopt 
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=5,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [122]:
# ! Important: We can use keras Tuner too for the hyperparameter search
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [123]:
mlflow.set_experiment("wine-quality")

with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials() # To perform the hyperparameter tunning. Instanciate a record of all the trials
    best=fmin(
        fn=objective, # Function to minimize. In this case use the objective function create who calls the train_model which use MAE
        space=space, # The params to search for the optimization
        algo=tpe.suggest, #! Optimization algorithm. In this case is the tpe (Tree-structured Parzen). We can choose betwen: tpe, randomsearch and adaptive tpe 
        max_evals=4, # Maximum trials to make
        trials=trials # Trials object to store the results for each evaluation
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"], reverse = False)[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(model = best_run["model"],
                                artifact_path = "model",
                                signature=signature,
                                input_example = train_x)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/03/05 20:13:50 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/5                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 25s 278ms/step - loss: 5.8575 - root_mean_squared_error: 5.9482
87/92 ━━━━━━━━━━━━━━━━━━━━ 0s 589us/step - loss: 2.6207 - root_mean_squared_error: 3.2501 
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5444 - root_mean_squared_error: 3.1784 - val_loss: 0.6669 - val_root_mean_squared_error: 0.8912

Epoch 2/5                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.5339 - root_mean_squared_error: 0.6512
88/92 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - loss: 0.6367 - root_mean_squared_error: 0.8135
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6357 - root_mean_squared_error: 0.8128 - val_loss: 0.6136 - val_root_mean_squared_error: 0.8084

Epoch 3/5                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.5784 - root_mean_squared_error: 0.6855
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - loss: 0.5922 - root_mean_squared_error: 0.7660
92/

c:\Anaconda\envs\mlops_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

 1/92 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step

92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step
Best parameters: {'lr': np.float64(0.022293094086829033), 'momentum': np.float64(0.9135408910090869)}
Best eval rmse: 0.7488476037979126


In [ ]:
## Inferencing
from mlflow.models import validate_serving_input

model_uri = 'runs:/9e0b52639ab44e6a864f5bd2f460fe42/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

e:\UDemy Final\MlFlowStarter\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[4.2569876],
       [7.2708473],
       [5.690359 ],
       ...,
       [6.881361 ],
       [6.6991577],
       [4.748927 ]], dtype=float32)

In [127]:
import mlflow

model_uri = 'runs:/8d57efee15fb4ee687871ae2c7ca43e8/model'
# This is the input example logged with the model
pyfunc_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame with the model and X_test data
import pandas as pd
pyfunc_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step


array([[5.9094925],
       [6.6082964],
       [6.533914 ],
       ...,
       [6.166654 ],
       [6.6686316],
       [5.7888823]], dtype=float32)

In [128]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1741202708431, current_stage='None', description=None, last_updated_timestamp=1741202708431, name='wine-quality', run_id='8d57efee15fb4ee687871ae2c7ca43e8', run_link=None, source='file:///c:/Users/Alejandro/Desktop/Proyectos_personales_github/Data-Science/04_MLOPS_concepts/01_MLFlow/02_DL_project/mlruns/883305291245152872/8d57efee15fb4ee687871ae2c7ca43e8/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>